In [ ]:
%env CUDA_VISIBLE_DEVICES=7
%env TRANSFORMERS_CACHE=/mnt/LLM/hub

%env HF_HOME=/mnt/LLM
%env OMP_NUM_THREADS=16

import os
import sys
sys.path.insert(0, '..')

import math
from argparse import Namespace
from typing import Sequence, Optional, List, Tuple

import torch
import torch.nn as nn
import transformers
from tqdm import trange

from prekv import datautils, modelutils
from prekv.quantizers import QuantizerBase, HiggsQuantizer
from prekv.linear_utils import fit_linear_regression
from train_predictors import OutputCatcher, get_predictor, get_dequant_values, compute_relative_mse
from ppl import evaluate_perplexity
from datasets import load_dataset
from prekv.cache_utils import TreatPrefixSeparately,PredictorHiggsCache,SingleChunkQuantizedCacheWithPredictors
from functools import partial

In [ ]:
def make_arg_parser():
    import argparse

    parser = argparse.ArgumentParser(add_help=True)

    parser.add_argument(
        "--model_name",
        default = "unsloth/Llama-3.2-3B",
        type=str,
        help="path to llama model to load, as in LlamaForCausalLM.from_pretrained()",
    )
    parser.add_argument(
        "--dataset",
        default="pajama",
        type=str,
        help="Dataset name [c4, pajama] or path to data where to extract calibration data from.",
    )
    parser.add_argument(
        "--torch_dtype",
        type=str,
        default="auto",
        choices=["auto", "float16", "float32", "bfloat16"],
        help="dtype to load the model in",
    )
    parser.add_argument(
        "--compute_dtype",
        type=str,
        default=None,
        help="dtype for computing activations",
    )
    parser.add_argument(
        "--model_seqlen",
        type=int,
        default=8192,
        help="Model seqlen and calibration data context length.",
    )
    parser.add_argument("--devices",
                        metavar="N",
                        type=str,
                        nargs="+",
                        default=None,
                        help="List of devices")
    parser.add_argument(
        "--seed",
        type=int,
        default=0,
        help="Seed for calibration data and initialization. "
             "Note that the main training is not strictly deterministic.",
    )
    parser.add_argument(
        "--offload_activations",
        action="store_true",
        help="Offload activations to RAM to save GPU memory.",
    )

    parser.add_argument(
        "--total_nsamples",
        type=int,
        default=256,
        help="Number of calibration data samples.If None take all calibration data.",
    )
    parser.add_argument(
        "--valid_nsamples",
        type=int,
        default=32,
        help="Number of calibration data samples.If None take all calibration data.",
    )
    parser.add_argument(
        "--chunk_size",
        type=int,
        default=4096,
        help="Number of tokens in one chunk.",
    )

    parser.add_argument(
        "--percdamp",
        type=float,
        default=1e-3,
        help="Percent of the average Hessian diagonal to use for dampening.",
    )

    parser.add_argument(
        "--hadamard_groupsize",
        type=int,
        default=1024,
        help="Groupsize of Hadamard transform for HIGGS.",
    )
    parser.add_argument(
        "--edenn_d",
        type=int,
        default=6,
        help="The grid dimension d for HIGGS.",
    )
    parser.add_argument(
        "--edenn_n",
        type=int,
        default=4096,
        help="The grid size n for HIGGS.",
    )
    
    parser.add_argument(
        "--ppl_chunk_size", #<- need to be renamed
        type=int,
        default=32,
        help="Number of tokens in one chunk.",
    )
    parser.add_argument(
        "--ppl_buffer_size",#<- need to be renamed
        type=int,
        default=128,
        help="Number of tokens in one chunk.",
    )
    parser.add_argument(
        "--output_path",
        type=str,
        default="./key_value_predictors.pt",
        help="Path to save trained predictors for Key and Values",
    )

    parser.add_argument("--wandb", action="store_true", help="Whether to use wandb or store locally.")
    return parser

### Parsing Arguments

In [ ]:
parser = make_arg_parser()
torch.set_num_threads(min(16, torch.get_num_threads()))
args = parser.parse_args(args=[])
args.offload_activations = True

### Training from the scratch 

In [ ]:
# infer defaults
if args.devices is None:
    if torch.cuda.is_available():
        args.devices = [torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    else:
        args.devices = [torch.device("cpu")]
else:
    args.devices = [torch.device(device_str) for device_str in args.devices]
assert len(args.devices) == 1, "parallelism is still WIP"

# load model and data
model = transformers.AutoModelForCausalLM.from_pretrained(
    args.model_name, torch_dtype=args.torch_dtype, low_cpu_mem_usage=True, use_cache=False
)

data = datautils.get_loaders(
    args.dataset,
    nsamples=args.total_nsamples,
    seed=args.seed,
    model_path=args.model_name,
    seqlen=args.model_seqlen,
)

quantizer = HiggsQuantizer(args.hadamard_groupsize, args.edenn_d, args.edenn_n)

# Calibration: propagate a set of inputs through one layer at a time, train predictors as we go
layers = modelutils.get_layers(model)

inps, forward_args = modelutils.get_inps(
    model, data, args.model_seqlen, args.devices, args.offload_activations)

for k, v in forward_args.items():
    forward_args[k] = v.to(args.devices[0]) if isinstance(v, torch.Tensor) else v

outs = [torch.zeros_like(inp_tensor, pin_memory=inp_tensor.is_pinned()) for inp_tensor in inps]
old_attn_keys = None
old_attn_values = None

key_predictors = {}
value_predictors = {}

for layer_index in range(len(layers)):
    print(f"\n---------------- Layer {layer_index} of {len(layers)} ----------------")
    layer_device_original = next(layers[layer_index].parameters()).device
    layer_dtype_original = next(layers[layer_index].parameters()).dtype
    layer = layers[layer_index].to(device=args.devices[0], dtype=args.compute_dtype or layer_dtype_original)

    layer.self_attn.k_proj = OutputCatcher(layer.self_attn.k_proj, args.offload_activations)
    layer.self_attn.v_proj = OutputCatcher(layer.self_attn.v_proj, args.offload_activations)

    modelutils.update_outs_inplace_(args.devices, layer, inps, outs, **forward_args, compute_mse=False)

    attn_keys = layer.self_attn.k_proj.outputs
    assert all(elem.shape[0] == 1 for elem in attn_keys)
    attn_keys = [elem[0] for elem in attn_keys]

    attn_values = layer.self_attn.v_proj.outputs
    assert all(elem.shape[0] == 1 for elem in attn_values)
    attn_values = [elem[0] for elem in attn_values]

    layer.self_attn.k_proj = layer.self_attn.k_proj.inner
    layer.self_attn.v_proj = layer.self_attn.v_proj.inner

    layers[layer_index] = layer.to(device=layer_device_original, dtype=layer_dtype_original)
    del layer
    torch.cuda.empty_cache()

    inps, outs = outs, inps

    if layer_index == 0:
        old_attn_keys = attn_keys
        old_attn_values = attn_values
        continue

    ### training predictor below ###
    key_predictor_inputs = list(old_attn_keys)
    key_predictor, mse_train_keys, mse_valid_keys = get_predictor(args, key_predictor_inputs, attn_keys)
    attn_keys = get_dequant_values(args, quantizer, key_predictor, key_predictor_inputs, attn_keys)
    del key_predictor_inputs
    key_predictors[layer_index] = key_predictor.cpu()
    train_bits_keys = - math.log(mse_train_keys) / math.log(4)
    valid_bits_keys = - math.log(mse_valid_keys) / math.log(4)
    print(f'{layer_index=}\tPREDICTOR_KEYS   \t| relMSE train: {mse_train_keys:.4f} valid: {mse_valid_keys:.4f} '
          f'| equiv.bits train: {train_bits_keys:.2f} valid: {valid_bits_keys:.2f}')
    value_predictor_inputs = [
        torch.cat([k_i, old_v_i], dim=-1) for k_i, old_v_i in zip(attn_keys, old_attn_values)]
    value_predictor, mse_train_values, mse_valid_values = get_predictor(args, value_predictor_inputs, attn_values)
    attn_values = get_dequant_values(args, quantizer, value_predictor, value_predictor_inputs, attn_values)
    value_predictors[layer_index] = value_predictor.cpu()
    del value_predictor_inputs
    train_bits_values = - math.log(mse_train_values) / math.log(4)
    valid_bits_values = - math.log(mse_valid_values) / math.log(4)
    print(
        f'{layer_index=}\tPREDICTOR_VALUES \t| relMSE train: {mse_train_values:.4f} valid: {mse_valid_values:.4f} '
        f'| equiv.bits train: {train_bits_values:.2f} valid: {valid_bits_values:.2f}')

    old_attn_keys, old_attn_values = attn_keys, attn_values

torch.save(dict(key_predictors=key_predictors, value_predictors=value_predictors), args.output_path)
print("Saved predictors to", args.output_path)

## PPL evaluation 

In [ ]:
with torch.no_grad():
    model.to(args.devices[0])
    testdata = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    config = transformers.AutoConfig.from_pretrained(args.model_name)
    tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name, config=config, padding_side="left")
    
    testenc = tokenizer("\n\n".join(testdata["text"]), return_tensors="pt")['input_ids']
    step_size = args.ppl_chunk_size

In [ ]:
with torch.no_grad():
    cache_factory = None
    ppl = evaluate_perplexity(model, testenc, args.model_seqlen, device=args.devices[0], step_size=step_size, cache_factory=cache_factory)

In [ ]:
with torch.no_grad():
    quantizer = HiggsQuantizer(args.hadamard_groupsize, args.edenn_d, args.edenn_n)
    cache_factory = lambda: TreatPrefixSeparately(prefix_size=4,
                          prefix_cache=transformers.DynamicCache(),
                          suffix_cache=PredictorHiggsCache(
                          config=model.config, min_buffer_size=args.ppl_buffer_size, save_dequantized_values=True,
                          make_quantized_cache=partial(
                                SingleChunkQuantizedCacheWithPredictors, quantizer=quantizer,
                                key_predictors=key_predictors, value_predictors=value_predictors
                            )
                        ))

    ppl_quantized = evaluate_perplexity(model, testenc, args.model_seqlen, device=args.devices[0], step_size=step_size, cache_factory=cache_factory)

In [ ]:
print(f"PPL on with static cache {ppl}\nPPL on with quantized cache {ppl_quantized}\n")